## SageMaker Experiment Management  - Python SDK API Walkthrough

This sample notebook demonstrates the basic usage of SageMaker Experiment Management Python SDK API.

### Setup

In [1]:
!aws configure add-model --service-model file://./source/build/model/sagemaker-2017-07-24.normal.json --service-name sagemaker
!cp ./source/build/model/sagemaker-2017-07-24.paginators.json ~/.aws/models/sagemaker/2017-07-24/paginators.json
!cp ./source/build/model/sagemaker-2017-07-24.waiters-2.json ~/.aws/models/sagemaker/2017-07-24/waiters-2.json

Traceback (most recent call last):
  File "C:\Python27\Scripts\aws.cmd", line 50, in <module>
    import awscli.clidriver
  File "C:\Users\212757215\AppData\Roaming\Python\Python37\site-packages\awscli\clidriver.py", line 17, in <module>
    import botocore.session
ModuleNotFoundError: No module named 'botocore'
'cp' is not recognized as an internal or external command,
operable program or batch file.
'cp' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import time
import boto3

import sagemaker
from sagemaker.experiments.experiment import Experiment
from sagemaker.experiments.trial import Trial
from sagemaker.experiments.trial_component import TrialComponent

ModuleNotFoundError: No module named 'boto3'

In [ ]:
sess = boto3.Session()
role = sagemaker.get_execution_role()

### Python SDK API usage

`Create`

In [ ]:
try:
    test_experiment = Experiment.create(
        experiment_name="test-experiment", 
        description="test-experiment")
    print(test_experiment)
except Exception as e:
    print(e)

In [ ]:
try:
    test_trial = test_experiment.create_trial(trial_name="test-trial")
    print(test_trial)
except Exception as e:
    print(e)

In [ ]:
# create a trial tracker, a trial component will be created automatically
# and we can log parameters and artifacts through tracker
with test_trial.create_tracker(component_name="test-component") as test_tracker:
    test_tracker.log_parameter(name='epoch', value=10)
    test_tracker.log_parameters({
        'layer': 2,
        'batch_size': 128,
        'optimizer': 'sgd',
    })
    test_tracker.log_input(name="training-set", value="s3://my-test-dataset")
    
print(test_tracker.trial_component_obj)

`Describe/Load & List`

In [ ]:
# to load an experiment created before
try:
    loaded_experiment = Experiment.load(experiment_name="test-experiment")
    print(loaded_experiment)
except Exception as e:
    print(e)

In [ ]:
# list trial summary
for loaded_trial_summary in loaded_experiment.list_trials():
    print(loaded_trial_summary)

# load the trial
loaded_trial = Trial.load(trial_name=loaded_trial_summary.trial_name)

`Update`

In [ ]:
# update the experiment description
loaded_experiment.description = "updated exp description."
try:
    loaded_experiment.update()
except Exception as e:
    print(e)

# load it again and check if update succeeds
print(Experiment.load(experiment_name="test-experiment").description)

In [ ]:
# let's create another experiment
try:
    test_experiment_2 = Experiment.create(
        experiment_name="test-experiment-2", 
        description="another experiment")
    print(test_experiment_2)
except Exception as e:
    print(e)

In [ ]:
# move the trial to the new experiment
loaded_trial.experiment_name = "test-experiment-2"
try:
    loaded_trial.update()
except Exception as e:
    print(e)
    
# load it again and check if update succeeds
print(Trial.load(trial_name="test-trial").experiment_name)

`Delete`

In [ ]:
# you cannot delete a trial with trial component inside.
try:
    loaded_trial.delete()
except Exception as e:
    print(e)

In [ ]:
# you cannot deleted an experiment with trial inside.
try:
    test_experiment_2.delete()
except Exception as e:
    print(e)

In [ ]:
# since test-trial is moved to test-experiment-2, test-experiment is empty, thus, it can be deleted.
try:
    test_experiment.delete()
except Exception as e:
    print(e)

In [ ]:
def cleanup(experiment):
    '''Clean up everything in the given experiment object'''
    for trial_summary in experiment.list_trials():
        trial = Trial.load(trial_name=trial_summary.trial_name)
        for trial_step in trial.list_trial_components():
            trial_step.delete()
            # to prevent throttling
            time.sleep(1)
        trial.delete()
    
    experiment.delete()

In [ ]:
cleanup(test_experiment_2)
del test_experiment_2